# Palace Python Interface Tutorial

This notebook demonstrates how to use Palace through its Python interface for electromagnetic simulations.

## Overview

Palace is a 3D finite element solver for computational electromagnetics. The Python interface provides:
- Easy configuration management
- Programmatic simulation control
- Result post-processing utilities
- Integration with Python scientific stack

In [ ]:
# Import necessary libraries
import json
import sys

import matplotlib.pyplot as plt
import numpy as np

# Import Palace Python interface
# Note: Adjust path as needed for your installation
sys.path.append("..")
from palace import PalaceSolver
from palace.utils import (
    create_basic_config,
    save_config,
)

# Set up matplotlib for inline plotting
%matplotlib inline
plt.style.use("default")
plt.rcParams["figure.figsize"] = (10, 6)

## 1. Getting Started - Check Palace Installation

In [ ]:
# Check if Palace is available
try:
    solver = PalaceSolver()
    print(f"✓ Palace found at: {solver.executable}")
    palace_available = True
except Exception as e:
    print(f"⚠ Palace not found: {e}")
    print("Some examples will run in demonstration mode only.")
    palace_available = False

## 2. Creating Configuration Files

Palace uses JSON configuration files to define simulations. Let's create configurations for different types of simulations.

In [ ]:
# Create a basic eigenmode configuration
mesh_file = "example_mesh.msh"  # Replace with actual mesh file

eigenmode_config = create_basic_config(mesh_file, "Eigenmode")

# Customize the configuration
eigenmode_config["Solver"]["Eigenmode"].update(
    {
        "Target": 5.0e9,  # Target frequency: 5 GHz
        "Tol": 1e-9,  # Tolerance
        "MaxIts": 1000,  # Maximum iterations
        "MaxSize": 20,  # Maximum eigenspace size
    }
)

# Add material properties
eigenmode_config["Model"]["Domain"] = [
    {
        "Index": 1,
        "Material": {
            "Permeability": 1.0,
            "Permittivity": 9.8,  # Silicon relative permittivity
            "LossTan": 1e-4,  # Loss tangent
        },
    }
]

# Save configuration
save_config(eigenmode_config, "eigenmode_example.json")
print("✓ Created eigenmode configuration")

# Display the configuration
print("\nConfiguration preview:")
print(json.dumps(eigenmode_config, indent=2)[:500] + "...")

In [ ]:
# Create a driven simulation configuration
driven_config = create_basic_config(mesh_file, "Driven")

# Customize frequency sweep
driven_config["Solver"]["Driven"].update(
    {
        "MinFreq": 1.0e9,  # 1 GHz
        "MaxFreq": 10.0e9,  # 10 GHz
        "FreqStep": 0.1e9,  # 100 MHz steps
    }
)

# Add port boundary conditions
driven_config["Model"]["Boundary"] = [
    {
        "Index": 1,
        "LumpedPort": {
            "R": 50.0,  # 50 ohm impedance
            "Excitation": True,
        },
    },
    {"Index": 2, "LumpedPort": {"R": 50.0}},
]

save_config(driven_config, "driven_example.json")
print("✓ Created driven simulation configuration")
print(
    f"Frequency range: {driven_config['Solver']['Driven']['MinFreq'] / 1e9:.1f} - {driven_config['Solver']['Driven']['MaxFreq'] / 1e9:.1f} GHz"
)

## 3. Running Simulations

There are multiple ways to run Palace simulations through Python:

In [ ]:
# Method 1: Simple function call
if palace_available:
    print("Running simulation using convenience function...")

    # This would run the simulation if we had a valid mesh file
    # result = run_palace("eigenmode_example.json", num_procs=2)

    print("✓ (Simulation would run here with valid mesh file)")
else:
    print("Palace not available - showing how simulation would be called:")
    print("result = run_palace('eigenmode_example.json', num_procs=2)")

In [ ]:
# Method 2: Using PalaceSolver class for more control
if palace_available:
    solver = PalaceSolver()

    # Validate configuration first
    if solver.validate_config("driven_example.json"):
        print("✓ Configuration is valid")

        # Run with custom settings
        # result = solver.run(
        #     "driven_example.json",
        #     output_dir="./results",
        #     num_procs=4
        # )

        print("✓ (Simulation would run here)")
    else:
        print("✗ Configuration validation failed")
else:
    print("Demonstration of solver class usage:")
    print("solver = PalaceSolver()")
    print("result = solver.run(config_file, output_dir='./results')")

## 4. Post-Processing Results

Palace generates CSV files with simulation results. Let's create some example data and show how to process it:

In [ ]:
# Create example eigenmode results
print("Creating example eigenmode results...")

# Simulate eigenfrequency results
eigenfreqs = np.array([4.87e9, 5.23e9, 6.14e9, 7.89e9, 8.76e9])  # Hz
q_factors = np.array([15000, 12000, 18000, 8500, 11000])

# Save as CSV
eig_data = np.column_stack([range(1, len(eigenfreqs) + 1), eigenfreqs, q_factors])
np.savetxt(
    "example_eigenvalues.csv",
    eig_data,
    delimiter=",",
    header="Mode,Frequency(Hz),Q_Factor",
    fmt="%d,%.6e,%.1f",
)

# Display results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(range(1, len(eigenfreqs) + 1), eigenfreqs / 1e9)
plt.xlabel("Mode Number")
plt.ylabel("Frequency (GHz)")
plt.title("Eigenmode Frequencies")
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.bar(range(1, len(q_factors) + 1), q_factors)
plt.xlabel("Mode Number")
plt.ylabel("Q Factor")
plt.title("Quality Factors")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Found {len(eigenfreqs)} eigenmodes")
print(f"Frequency range: {eigenfreqs[0] / 1e9:.2f} - {eigenfreqs[-1] / 1e9:.2f} GHz")
print(f"Average Q factor: {np.mean(q_factors):.0f}")

In [ ]:
# Create example S-parameter results
print("Creating example S-parameter results...")

# Generate frequency sweep data
freq = np.linspace(1e9, 10e9, 91)  # 1-10 GHz, 100 MHz steps

# Simulate S-parameters for a resonant structure
f0 = 5.5e9  # Resonant frequency
Q = 100  # Quality factor

# S11 (reflection)
s11 = (1j * 2 * Q * (freq - f0) / f0) / (1 + 1j * 2 * Q * (freq - f0) / f0)

# S21 (transmission)
s21 = 1 / (1 + 1j * 2 * Q * (freq - f0) / f0)

# Save S-parameters
s_data = np.column_stack([freq, np.abs(s11), np.angle(s11), np.abs(s21), np.angle(s21)])
np.savetxt(
    "example_sparameters.csv",
    s_data,
    delimiter=",",
    header="Frequency(Hz),|S11|,arg(S11),|S21|,arg(S21)",
)

# Plot results
plt.figure(figsize=(15, 10))

# S11 magnitude
plt.subplot(2, 2, 1)
plt.plot(freq / 1e9, 20 * np.log10(np.abs(s11)))
plt.xlabel("Frequency (GHz)")
plt.ylabel("|S11| (dB)")
plt.title("S11 Magnitude")
plt.grid(True)

# S11 phase
plt.subplot(2, 2, 2)
plt.plot(freq / 1e9, np.angle(s11) * 180 / np.pi)
plt.xlabel("Frequency (GHz)")
plt.ylabel("S11 Phase (degrees)")
plt.title("S11 Phase")
plt.grid(True)

# S21 magnitude
plt.subplot(2, 2, 3)
plt.plot(freq / 1e9, 20 * np.log10(np.abs(s21)))
plt.xlabel("Frequency (GHz)")
plt.ylabel("|S21| (dB)")
plt.title("S21 Magnitude")
plt.grid(True)

# Smith chart (S11)
plt.subplot(2, 2, 4)
theta = np.linspace(0, 2 * np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), "k-", alpha=0.3)  # Unit circle
plt.plot(np.real(s11), np.imag(s11), "b-", linewidth=2)
plt.plot(np.real(s11[0]), np.imag(s11[0]), "go", markersize=8, label="Start")
plt.plot(np.real(s11[-1]), np.imag(s11[-1]), "ro", markersize=8, label="End")
plt.xlabel("Real(S11)")
plt.ylabel("Imag(S11)")
plt.title("S11 on Smith Chart")
plt.axis("equal")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Find resonance
min_s11_idx = np.argmin(np.abs(s11))
resonant_freq = freq[min_s11_idx]
min_s11_db = 20 * np.log10(np.abs(s11[min_s11_idx]))

print(f"✓ Resonance found at {resonant_freq / 1e9:.3f} GHz")
print(f"Minimum |S11|: {min_s11_db:.1f} dB")
print(f"Maximum |S21|: {20 * np.log10(np.max(np.abs(s21))):.1f} dB")

## 5. Advanced Post-Processing

Let's demonstrate some advanced analysis techniques:

In [ ]:
# Quality factor extraction from S-parameters
def extract_q_factor(freq, s21):
    """Extract Q factor from S21 transmission data."""
    # Find peak (resonance)
    peak_idx = np.argmax(np.abs(s21))
    f0 = freq[peak_idx]
    peak_value = np.abs(s21[peak_idx])

    # Find -3dB points
    half_power = peak_value / np.sqrt(2)

    # Find frequencies where |S21| crosses half-power level
    left_idx = np.where((freq < f0) & (np.abs(s21) >= half_power))[0]
    right_idx = np.where((freq > f0) & (np.abs(s21) >= half_power))[0]

    if len(left_idx) > 0 and len(right_idx) > 0:
        f1 = freq[left_idx[0]]
        f2 = freq[right_idx[-1]]
        bandwidth = f2 - f1
        q_factor = f0 / bandwidth
        return q_factor, f0, bandwidth
    else:
        return None, f0, None


# Extract Q factor from our example data
q_extracted, f0_extracted, bw = extract_q_factor(freq, s21)

if q_extracted:
    print(f"Extracted Q factor: {q_extracted:.1f}")
    print(f"Resonant frequency: {f0_extracted / 1e9:.3f} GHz")
    print(f"3-dB bandwidth: {bw / 1e6:.1f} MHz")
    print(f"Expected Q factor: {Q}")
else:
    print("Could not extract Q factor from data")

In [ ]:
# Energy analysis for eigenmode simulations
def analyze_mode_energy(freq_list, q_list):
    """Analyze energy storage and loss in eigenmodes."""

    # Calculate stored energy (proportional to frequency squared)
    stored_energy = freq_list**2 / np.max(freq_list**2)  # Normalized

    # Calculate power loss (inversely proportional to Q)
    power_loss = freq_list / q_list
    power_loss = power_loss / np.max(power_loss)  # Normalized

    return stored_energy, power_loss


# Analyze our eigenmode data
stored_energy, power_loss = analyze_mode_energy(eigenfreqs, q_factors)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
modes = range(1, len(eigenfreqs) + 1)
width = 0.35
plt.bar(
    [m - width / 2 for m in modes],
    stored_energy,
    width,
    label="Stored Energy",
    alpha=0.8,
)
plt.bar(
    [m + width / 2 for m in modes], power_loss, width, label="Power Loss", alpha=0.8
)
plt.xlabel("Mode Number")
plt.ylabel("Normalized Energy")
plt.title("Energy Analysis")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(eigenfreqs / 1e9, q_factors, s=100, alpha=0.7)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Q Factor")
plt.title("Q Factor vs Frequency")
plt.grid(True, alpha=0.3)

# Add mode labels
for i, (f, q) in enumerate(zip(eigenfreqs, q_factors)):
    plt.annotate(
        f"Mode {i + 1}",
        (f / 1e9, q),
        xytext=(5, 5),
        textcoords="offset points",
        fontsize=9,
    )

plt.tight_layout()
plt.show()

print("Energy Analysis Results:")
for i, (f, q, se, pl) in enumerate(
    zip(eigenfreqs, q_factors, stored_energy, power_loss)
):
    print(
        f"Mode {i + 1}: f={f / 1e9:.2f} GHz, Q={q:.0f}, Energy={se:.3f}, Loss={pl:.3f}"
    )

## 6. Batch Processing and Automation

The Python interface makes it easy to run parameter sweeps and batch simulations:

In [ ]:
# Example: Parameter sweep over material properties
def create_parameter_sweep_configs():
    """Create multiple configurations for parameter sweep."""

    base_config = create_basic_config("example_mesh.msh", "Eigenmode")

    # Sweep over relative permittivity
    permittivity_values = [2.0, 4.0, 9.8, 11.7]  # Different materials
    material_names = ["PTFE", "Quartz", "Silicon", "GaAs"]

    configs = []

    for i, (eps_r, mat_name) in enumerate(zip(permittivity_values, material_names)):
        config = base_config.copy()
        config["Model"]["Domain"] = [
            {
                "Index": 1,
                "Material": {
                    "Permeability": 1.0,
                    "Permittivity": eps_r,
                    "LossTan": 1e-4,
                },
            }
        ]

        config_file = f"sweep_{mat_name.lower()}_config.json"
        save_config(config, config_file)
        configs.append((config_file, mat_name, eps_r))

    return configs


# Create sweep configurations
sweep_configs = create_parameter_sweep_configs()

print("Created parameter sweep configurations:")
for config_file, material, eps_r in sweep_configs:
    print(f"  {material} (εᵣ={eps_r}): {config_file}")

# Simulate running the sweep (would actually run if Palace available)
print("\nSimulation sweep (demonstration):")
simulated_results = []

for config_file, material, eps_r in sweep_configs:
    # Simulate frequency scaling with permittivity: f ∝ 1/√εᵣ
    f0_vacuum = 5e9  # Reference frequency
    scaled_freq = f0_vacuum / np.sqrt(eps_r)

    simulated_results.append((material, eps_r, scaled_freq))
    print(f"  {material}: f₀ ≈ {scaled_freq / 1e9:.2f} GHz")

# Plot results
materials, eps_values, frequencies = zip(*simulated_results)

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.bar(materials, frequencies)
plt.ylabel("Resonant Frequency (Hz)")
plt.title("Frequency vs Material")
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.loglog(eps_values, frequencies, "o-")
plt.xlabel("Relative Permittivity")
plt.ylabel("Resonant Frequency (Hz)")
plt.title("Frequency Scaling Law")
plt.grid(True)

# Add theoretical f ∝ 1/√εᵣ line
eps_theory = np.logspace(0, 1.2, 50)
f_theory = f0_vacuum / np.sqrt(eps_theory)
plt.loglog(eps_theory, f_theory, "r--", alpha=0.7, label="f ∝ 1/√εᵣ")
plt.legend()

plt.tight_layout()
plt.show()

## 7. Summary and Next Steps

This notebook demonstrated:

1. **Configuration Management**: Creating and customizing Palace configuration files
2. **Simulation Control**: Running simulations programmatically
3. **Result Processing**: Reading and analyzing CSV output files
4. **Visualization**: Plotting S-parameters, eigenmodes, and Smith charts
5. **Advanced Analysis**: Q-factor extraction and energy analysis
6. **Automation**: Parameter sweeps and batch processing

### Next Steps:

- Install Palace following the [official documentation](https://awslabs.github.io/palace/)
- Try running with real mesh files from the `examples/` directory
- Explore more advanced configuration options
- Integrate with other Python libraries (scipy, pandas, etc.)
- Create custom post-processing workflows for your specific applications

### Useful Resources:

- [Palace Documentation](https://awslabs.github.io/palace/)
- [Palace GitHub Repository](https://github.com/awslabs/palace)
- [MFEM Library](https://mfem.org/) (underlying finite element library)
- Palace configuration schema in `scripts/schema/config-schema.json`

In [ ]:
# Clean up example files
import glob

example_files = (
    glob.glob("example_*.json") + glob.glob("example_*.csv") + glob.glob("sweep_*.json")
)
print(f"Generated {len(example_files)} example files:")
for f in example_files:
    print(f"  {f}")

print(
    "\n✓ Tutorial completed! Check the generated files and try modifying them for your own simulations."
)